In [1]:
import numpy as np
import faiss
import time
import pandas as pd
from os import path

In [2]:
ngpus = faiss.get_num_gpus()
print("number of GPUs:", ngpus)

number of GPUs: 1


## Configuration Variables

In [3]:
root = 'retrieval_array_path/'
model = "mrl/" # mrl/, mrl_e/, ff/, slimmable/
dataset = '1K' # 1K, 4K, V2
index_type = 'exactl2' # exactl2, hnsw8, hnsw32
hnsw_max_neighbors = 32 # 8, 32
k = 2048 # shortlist length, default set to max supported by FAISS
nesting_list = [8, 16, 32, 64, 128, 256, 512, 1024, 2048]

## FAISS Index Building and NN Search

In [4]:
if model == 'mrl/':
    config = 'mrl1_e0_ff2048'
elif model == 'mrl_e/':
    config = 'mrl0_e1_ff2048'
elif model == 'ff/':
    config = 'mrl0_e0_ff16'
else: 
    raise Exception("Unsupported pretrained model.")
    
db_csv = dataset + '_train_' + config + '-X.npy'
query_csv = dataset + '_val_' + config + '-X.npy'

# ImageNetv2 is only a test set; set database to ImageNet-1K
if dataset == 'V2':
    db_csv = '1K_train_' + config + '-X.npy'

In [5]:
start = time.time()
database = np.load(root+db_csv)
queryset = np.load(root+query_csv)
print("CSV file load time= ", time.time() - start)

CSV file load time=  4.611276626586914


In [6]:
if index_type == 'exactl2':
    use_gpu = 1 # GPU inference for exact search
else:
    use_gpu = 0 # GPU inference for HNSW is currently not supported by FAISS

In [11]:
for retrieval_dim in nesting_list:
    index_file = root+'index_files/'+model+'/'+dataset+'_'+str(retrieval_dim)+'_'+index_type+'.index'
    
    # Load or build index
    if path.exists(index_file):
        print("Loading index file: " + index_file)
        cpu_index = faiss.read_index(index_file)

    else:
        print("Generating index file: " + index_file)
        
        xb = np.ascontiguousarray(np.load(root+db_csv)[:,:retrieval_dim], dtype=np.float32)
        faiss.normalize_L2(xb)
        d = xb.shape[1]                           # dimension
        nb = xb.shape[0]                       # database size
        print("database: ", xb.shape)

        start = time.time()
        if index_type == 'exactl2':
            print("Building Exact L2 Index")
            cpu_index = faiss.IndexFlatL2(d)   # build the index
        else:
            print("Building HNSW%d Index" % hnsw_max_neighbors)
            cpu_index = faiss.IndexHNSWFlat(d, hnsw_max_neighbors)
        cpu_index.add(xb)                  # add vectors to the index
        faiss.write_index(cpu_index, index_file)
        print("GPU Index build time= ", time.time() - start)
    
    if use_gpu:
        index = faiss.index_cpu_to_all_gpus(
            cpu_index
        )
    else:
        index = cpu_index
    
    # Load the queries
    xq = np.ascontiguousarray(np.load(root+query_csv)[:,:retrieval_dim], dtype=np.float32)
    faiss.normalize_L2(xq)
    nq = xq.shape[0]
    print("queries: ", xq.shape)

    start = time.time()
    D, I = index.search(xq, k)
    end = time.time() - start
    print("GPU %d-NN search time= %f" % (k, end))
    
    start = time.time()
    nn_dir = root+"neighbors/"+model+'/'+index_type+"_"+str(retrieval_dim)+"dim_"+str(k)+"shortlist_"+dataset+".csv"
    pd.DataFrame(I).to_csv(nn_dir, header=None, index=None)
    end = time.time() - start
    print("NN file write time= %f\n" % (end))


Loading index file: ../inference/index_files/mrl/1K_8_exactl2.index
queries:  (50000, 8)
GPU 2048-NN search time= 5.033833
NN file write time= 26.747888

Loading index file: ../inference/index_files/mrl/1K_16_exactl2.index
queries:  (50000, 16)
GPU 2048-NN search time= 5.044729
NN file write time= 27.012290

Loading index file: ../inference/index_files/mrl/1K_32_exactl2.index
queries:  (50000, 32)
GPU 2048-NN search time= 5.224507
NN file write time= 26.960253

Loading index file: ../inference/index_files/mrl/1K_64_exactl2.index
queries:  (50000, 64)
GPU 2048-NN search time= 4.217198
NN file write time= 26.806727

Loading index file: ../inference/index_files/mrl/1K_128_exactl2.index
queries:  (50000, 128)
GPU 2048-NN search time= 4.698427
NN file write time= 26.872504

Loading index file: ../inference/index_files/mrl/1K_256_exactl2.index
queries:  (50000, 256)
GPU 2048-NN search time= 5.330067
NN file write time= 26.814057

Loading index file: ../inference/index_files/mrl/1K_512_exactl